In [27]:
import pandas as pd

In [28]:
data = pd.read_parquet('../data/tokyo-clean.parquet')

In [29]:
data['Municipality'].unique()

array(['千代田区 (Chiyoda Ward)', '中央区 (Chuo Ward)', '港区 (Minato Ward)',
       '新宿区 (Shinjuku Ward)', '文京区 (Bunkyo Ward)', '台東区 (Taito Ward)',
       '墨田区 (Sumida Ward)', '江東区 (Koto Ward)', '品川区 (Shinagawa Ward)',
       '目黒区 (Meguro Ward)', '大田区 (Ota Ward)', '世田谷区 (Setagaya Ward)',
       '渋谷区 (Shibuya Ward)', '中野区 (Nakano Ward)', '杉並区 (Suginami Ward)',
       '豊島区 (Toshima Ward)', '北区 (Kita Ward)', '荒川区 (Arakawa Ward)',
       '板橋区 (Itabashi Ward)', '練馬区 (Nerima Ward)', '足立区 (Adachi Ward)',
       '葛飾区 (Katsushika Ward)', '江戸川区 (Edogawa Ward)',
       '八王子市 (Hachioji City)', '立川市 (Tachikawa City)',
       '武蔵野市 (Musashino City)', '三鷹市 (Mitaka City)', '青梅市 (Oume City)',
       '府中市 (Fuchu City)', '昭島市 (Akishima City)', '調布市 (Chofu City)',
       '町田市 (Machida City)', '小金井市 (Koganei City)', '小平市 (Kodaira City)',
       '日野市 (Hino City)', '東村山市 (Higashimurayama City)',
       '国分寺市 (Kokubunji City)', '国立市 (Kunitachi City)',
       '福生市 (Fussa City)', '狛江市 (Komae City)',
       '東大和市 (Higas

In [ ]:
# adding new feature - ward flag

# Create the binary flag for 23 Special Wards
# 1 = Central Tokyo (Ward), 0 = Western Tokyo/Islands (City/Town/Village)
data['Is_Ward'] = data['Municipality'].str.contains('区', case=False).astype(int)

In [32]:
# adding new feature - building age at the time of the transaction

data['BuildingAge'] = data['TransactionYear'] - data['BuildingYear']

In [ ]:
def parse_floor_plan(df):
    """
    Parses Japanese real estate 'LDK' strings into numerical features.
    Example: '2LDK+S' -> RoomCount:2, L:1, D:1, K:1, S:1
    """
    # Avoid SettingWithCopy warnings by operating on a copy
    df = df.copy()

    # 1. Standardize special text cases
    # 'Studio Apartment' and 'Open Floor' are effectively 1-room layouts
    # We map them to '1R' so our regex can pick up the '1'
    # Has_L, Has_D, Has_K will allow XGBoost to identify the studio apartments
    # Area will allow XGBoost to discriminate between Studio Apartment and Open Floor
    df['TempPlan'] = df['FloorPlan'].replace({
        'Studio Apartment': '1R',
        'Open Floor': '1R',
        'Duplex': '2LDK',  # Assumption: Treat Duplex as at least 2 rooms
        'None': '0R'  # Handle missing strings if any
    })

    # 2. Extract Room Count
    # Regex: Look for the number at the very start of the string
    df['RoomCount'] = df['TempPlan'].str.extract(r'^(\d+)').fillna(0).astype(int)

    # 3. Create Boolean Flags for Layout Types
    # L = Living, D = Dining, K = Kitchen, S = Service Room (Storage)
    df['Has_L'] = df['TempPlan'].str.contains('L', case=False, na=False).astype(int)
    df['Has_D'] = df['TempPlan'].str.contains('D', case=False, na=False).astype(int)
    df['Has_K'] = df['TempPlan'].str.contains('K', case=False, na=False).astype(int)
    
    # 4. Handle the "Service Room" (Nando)
    # Usually denoted as "+S" or just "S"
    df['Has_S'] = df['TempPlan'].str.contains('S', case=False, na=False).astype(int)

    # 5. Clean up
    # TODO drop the original FloorPlan column as well in the script
    # we leave it here to illustrate the effect below
    # df = df.drop(columns=['TempPlan', 'FloorPlan'])
    df.drop('TempPlan', axis=1, inplace=True)
    
    return df

data.FloorPlan.unique()

array(['1K', '2LDK', None, '1LDK', '1DK', '1R', '3LDK', '3DK', '4LDK',
       '2DK', 'Open Floor', '2K', 'Studio Apartment', '1LDK+S', '3LK',
       'Duplex', '1K+S', '4LDK+S', '3LDK+S', '2LDK+S', '3K', '5LDK',
       '4DK', '3DK+S', '2DK+S', '2LK', '6LDK', '7LDK', '1DK+S', '1LK',
       '3LD', '1R+S', '4K', '4DK+S', '2LK+S', '2LD+S', '3LD+S', '2K+S',
       '5LDK+S', '2LD', '5DK', '1L+S', '6LDK+S', '3LDK+K', '1L', '6DK',
       '1LK+S', '8LDK', '5LK', '5K', '6DK+S', '7LDK+S', '3K+S', '7DK',
       '6K', '1LDK+K', '5K+S', '5DK+S', '3LK+S', '4K+S', '8LDK+S', '2L+S',
       '4LK', '6K+S', '1LD+S', '6LK', '4L+K'], dtype=object)

In [34]:
data = parse_floor_plan(data)
print(data[['FloorPlan', 'RoomCount', 'Has_L', 'Has_D', 'Has_K', 'Has_S']].head())
data.drop('FloorPlan', axis=1, inplace=True)

  FloorPlan  RoomCount  Has_L  Has_D  Has_K  Has_S
0        1K          1      0      0      1      0
1      2LDK          2      1      1      1      0
2      None          0      0      0      0      0
3      None          0      0      0      0      0
4      1LDK          1      1      1      1      0
